In [2]:
import sys
from Load import *
from UE import *
from Satellite import *
from Oracle import *
import time

def initial_assignment(UEss, Satss, oracle):
    C = UEss[0].coverage_info
    if oracle is None:
        # random assign
        for ueid in UEss:
            ue = UEss[ueid]
            possible_satellites = np.where(C[ueid, :, 0] == 1)[0]
            assert (len(possible_satellites) > 0)
            satid = random.choice(possible_satellites)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
    else:
        for ueid in UEss:
            ue = UEss[ueid]
            satid = oracle.query_init_satellite(ueid)
            assert(C[ueid, satid, 0] == 1)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
            
         


def monitor_timestamp(env):
    while True:
        if env.now % 100 == 0:
            print(f"Current simulation time {env.now}", file=sys.stderr)
        yield env.timeout(1)


if __name__ == "__main__":
    # configuration
    # When oracle_simulation is False, the oracle_assignment could be True/False
    # When oracle_simulation is True, the oracle_assignment must be True
    oracle_assignment = True
    oracle_simulation = True
    if oracle_simulation:
        assert(oracle_assignment)
    random.seed(10)

    # Loading scenarios
    beginning = time.time()
    UEs_template, satellites_template, C = load_scenario()
    DURATION = C.shape[2]
    env = simpy.Environment()

    assignment_oracle = None
    simulation_oracle = None
    if oracle_assignment:
        assignment_oracle = Oracle()
    if oracle_simulation:
        simulation_oracle = assignment_oracle
        
    UEs = {}
    for ue_template in UEs_template:
        ID = ue_template.ID
        if ID not in UEs:
            UEs[ID] = UE(
                identity=ID,
                position_x=ue_template.x,
                position_y=ue_template.y,
                coverage_info=C,
                oracle = simulation_oracle,
                env=env
            )
        else:
            print("ERROR")

    Satellites = {}
    for sat_template in satellites_template:
        ID = sat_template.ID
        if ID not in Satellites:
            Satellites[ID] = Satellite(
                identity=ID,
                position_x=sat_template.x,
                position_y=sat_template.y,
                height=sat_template.h,
                coverage_r=sat_template.r,
                velocity=sat_template.v,
                sind=sat_template.sind,
                cosd=sat_template.cosd,
                coverage_info=C,
                env=env
            )
        else:
            print("ERROR")

    for ueid in UEs:
        UEs[ueid].satellites = Satellites

    for satid in Satellites:
        Satellites[satid].UEs = UEs
        Satellites[satid].satellites = Satellites    
        
    print(f"Loading scenario in the simulation takes {time.time() - beginning}")
    # ========= UE, SAT objectives are ready =========
    initial_assignment(UEs, Satellites, assignment_oracle)

    env.process(monitor_timestamp(env))

    print('============= Experiment Log =============')
    env.run(until=DURATION)
    print('============= Experiment Ends =============')
    counter = allCounters(Satellites)
    # counter.generate_heap_map(100)


Loading scenario in the simulation takes 0.0685880184173584
============= Experiment Log =============
UE 0 deployed at time 0, positioned at (-65.49404166863776,-66.93099954936791)
UE 0 is served by Satellite 2
UE 1 deployed at time 0, positioned at (-287.73149927538367,-49.50814197968104)
UE 1 is served by Satellite 3
UE 2 deployed at time 0, positioned at (27.2424493687649,-49.68107255252102)
UE 2 is served by Satellite 0
UE 3 deployed at time 0, positioned at (-301.7140105153102,-71.38387128154312)
UE 3 is served by Satellite 2
UE 4 deployed at time 0, positioned at (21.202515821233824,-25.022899790455842)
UE 4 is served by Satellite 1
UE 5 deployed at time 0, positioned at (33.52408097445878,59.4355016396482)
UE 5 is served by Satellite 1
UE 6 deployed at time 0, positioned at (-336.1016244239552,-57.31796594972779)
UE 6 is served by Satellite 4
UE 7 deployed at time 0, positioned at (-72.4812937312826,-30.617001148024)
UE 7 is served by Satellite 2
UE 8 deployed at time 0, positi

Current simulation time 0
Current simulation time 100
Current simulation time 200
Current simulation time 300
Current simulation time 400
Current simulation time 500
Current simulation time 600
Current simulation time 700
Current simulation time 800
Current simulation time 900
Current simulation time 1000
Current simulation time 1100
Current simulation time 1200
Current simulation time 1300
Current simulation time 1400
Current simulation time 1500
Current simulation time 1600
Current simulation time 1700
Current simulation time 1800
Current simulation time 1900
Current simulation time 2000
Current simulation time 2100
Current simulation time 2200
Current simulation time 2300
Current simulation time 2400
Current simulation time 2500


UE 25 sends Satellite 1 the message {"task": "MEASUREMENT_REPORT", "candidates": [0, 2], "from": 25, "to": 1} at 2525
Satellite 1 sends Satellite 0 the message {"task": "HANDOVER_REQUEST", "ueid": 25, "from": 1, "to": 0} at 2525
Satellite 1 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 25, "from": 1, "to": 2} at 2525
Satellite 0 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 25, "condition": {"access_delay": 15, "ueid": 25, "source": 1, "satid": 0}, "from": 0, "to": 1} at 2525
Satellite 2 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 25, "condition": {"access_delay": 6, "ueid": 25, "source": 1, "satid": 2}, "from": 2, "to": 1} at 2525
Satellite 1 sends UE 25 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 15, "ueid": 25, "source": 1, "satid": 0}, {"access_delay": 6, "ueid": 25, "source": 1, "satid": 2}], "from": 1, "to": 25} at 2525
UE 25 sends Satellite 1 the message {"task": "RRC_RECONFIGURATI

Current simulation time 2600
Current simulation time 2700
Current simulation time 2800
Current simulation time 2900
Current simulation time 3000
Current simulation time 3100
Current simulation time 3200
Current simulation time 3300
Current simulation time 3400
Current simulation time 3500
Current simulation time 3600
Current simulation time 3700
Current simulation time 3800
Current simulation time 3900
Current simulation time 4000
Current simulation time 4100
Current simulation time 4200
Current simulation time 4300
Current simulation time 4400
Current simulation time 4500
Current simulation time 4600
Current simulation time 4700
Current simulation time 4800
Current simulation time 4900
Current simulation time 5000
Current simulation time 5100
Current simulation time 5200
Current simulation time 5300
Current simulation time 5400
Current simulation time 5500
Current simulation time 5600
Current simulation time 5700
Current simulation time 5800
Current simulation time 5900
Current simula

UE 46 sends Satellite 4 the message {"task": "MEASUREMENT_REPORT", "candidates": [5, 6], "from": 46, "to": 4} at 4922
Satellite 4 sends Satellite 5 the message {"task": "HANDOVER_REQUEST", "ueid": 46, "from": 4, "to": 5} at 4922
Satellite 4 sends Satellite 6 the message {"task": "HANDOVER_REQUEST", "ueid": 46, "from": 4, "to": 6} at 4922
Satellite 5 sends Satellite 4 the message {"task": "HANDOVER_RESPONSE", "ueid": 46, "condition": {"access_delay": 16, "ueid": 46, "source": 4, "satid": 5}, "from": 5, "to": 4} at 4922
Satellite 6 sends Satellite 4 the message {"task": "HANDOVER_RESPONSE", "ueid": 46, "condition": {"access_delay": 20, "ueid": 46, "source": 4, "satid": 6}, "from": 6, "to": 4} at 4922
Satellite 4 sends UE 46 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 16, "ueid": 46, "source": 4, "satid": 5}, {"access_delay": 20, "ueid": 46, "source": 4, "satid": 6}], "from": 4, "to": 46} at 4922
UE 46 sends Satellite 4 the message {"task": "RRC_RECONFIGURA

...

Current simulation time 10000
Current simulation time 10100
Current simulation time 10200
Current simulation time 10300
Current simulation time 10400
Current simulation time 10500
Current simulation time 10600
Current simulation time 10700
Current simulation time 10800
Current simulation time 10900
Current simulation time 11000
Current simulation time 11100
Current simulation time 11200
Current simulation time 11300
Current simulation time 11400
Current simulation time 11500
Current simulation time 11600
Current simulation time 11700
Current simulation time 11800
Current simulation time 11900
Current simulation time 12000
Current simulation time 12100
Current simulation time 12200
Current simulation time 12300
Current simulation time 12400
Current simulation time 12500
Current simulation time 12600
Current simulation time 12700
Current simulation time 12800
Current simulation time 12900
Current simulation time 13000
Current simulation time 13100
Current simulation time 13200
Current si

UE 32 sends Satellite 8 the message {"task": "MEASUREMENT_REPORT", "candidates": [9, 10], "from": 32, "to": 8} at 11932
Satellite 8 sends Satellite 9 the message {"task": "HANDOVER_REQUEST", "ueid": 32, "from": 8, "to": 9} at 11932
Satellite 8 sends Satellite 10 the message {"task": "HANDOVER_REQUEST", "ueid": 32, "from": 8, "to": 10} at 11932
Satellite 9 sends Satellite 8 the message {"task": "HANDOVER_RESPONSE", "ueid": 32, "condition": {"access_delay": 11, "ueid": 32, "source": 8, "satid": 9}, "from": 9, "to": 8} at 11932
Satellite 10 sends Satellite 8 the message {"task": "HANDOVER_RESPONSE", "ueid": 32, "condition": {"access_delay": 15, "ueid": 32, "source": 8, "satid": 10}, "from": 10, "to": 8} at 11932
Satellite 8 sends UE 32 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 11, "ueid": 32, "source": 8, "satid": 9}, {"access_delay": 15, "ueid": 32, "source": 8, "satid": 10}], "from": 8, "to": 32} at 11932
UE 32 sends Satellite 8 the message {"task": "RR

Current simulation time 14000
Current simulation time 14100
Current simulation time 14200
Current simulation time 14300
Current simulation time 14400
Current simulation time 14500
Current simulation time 14600
Current simulation time 14700
Current simulation time 14800
Current simulation time 14900
Current simulation time 15000
Current simulation time 15100
Current simulation time 15200
Current simulation time 15300
Current simulation time 15400
Current simulation time 15500
Current simulation time 15600
Current simulation time 15700
Current simulation time 15800
Current simulation time 15900
Current simulation time 16000
Current simulation time 16100
Current simulation time 16200
Current simulation time 16300
Current simulation time 16400
Current simulation time 16500
Current simulation time 16600
Current simulation time 16700
Current simulation time 16800
Current simulation time 16900
Current simulation time 17000
Current simulation time 17100
Current simulation time 17200
Current si

UE 33 sends Satellite 10 the message {"task": "MEASUREMENT_REPORT", "candidates": [11, 12], "from": 33, "to": 10} at 15133
Satellite 10 sends Satellite 11 the message {"task": "HANDOVER_REQUEST", "ueid": 33, "from": 10, "to": 11} at 15133
Satellite 10 sends Satellite 12 the message {"task": "HANDOVER_REQUEST", "ueid": 33, "from": 10, "to": 12} at 15133
Satellite 11 sends Satellite 10 the message {"task": "HANDOVER_RESPONSE", "ueid": 33, "condition": {"access_delay": 19, "ueid": 33, "source": 10, "satid": 11}, "from": 11, "to": 10} at 15133
Satellite 12 sends Satellite 10 the message {"task": "HANDOVER_RESPONSE", "ueid": 33, "condition": {"access_delay": 20, "ueid": 33, "source": 10, "satid": 12}, "from": 12, "to": 10} at 15133
Satellite 10 sends UE 33 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 19, "ueid": 33, "source": 10, "satid": 11}, {"access_delay": 20, "ueid": 33, "source": 10, "satid": 12}], "from": 10, "to": 33} at 15133
UE 33 sends Satellite 10 

Current simulation time 17400
Current simulation time 17500
Current simulation time 17600
Current simulation time 17700
Current simulation time 17800
Current simulation time 17900
Current simulation time 18000
Current simulation time 18100
Current simulation time 18200
Current simulation time 18300
Current simulation time 18400
Current simulation time 18500
Current simulation time 18600
Current simulation time 18700
Current simulation time 18800
Current simulation time 18900
Current simulation time 19000
Current simulation time 19100
Current simulation time 19200
Current simulation time 19300
Current simulation time 19400
Current simulation time 19500
Current simulation time 19600
Current simulation time 19700
Current simulation time 19800
Current simulation time 19900
